In [ ]:
!pip install transformers
!pip install pytorch-pretrained-bert
!pip install keybert
!pip install keybert[spacy]
!pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.8/123.8 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 32.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 40.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 43.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━

In [ ]:
from keybert import KeyBERT
import time

kw_model = KeyBERT()

In [13]:
# Стоп-слова в виде списка
stop_words_list = ["locate","identify", "spot", "find", "discover", "pinpoint","Locate","Identify","Spot","Find","Discover", "Pinpoint", "image", "Image", "Picture", "picture"]

In [78]:
start_time = time.time()
keywords = kw_model.extract_keywords(request, keyphrase_ngram_range=(1, 1), stop_words = stop_words_list)
end_time = time.time()
# Время в миллисекундах
execution_time = (end_time - start_time) * 1000
# Формирование строки с ключевыми словами, разделенными запятыми

filtered_keywords = [word for word, score in keywords if score > 0.35]

#keyword_string = ','.join([word for word, score in filtered_keywords])

In [79]:
filtered_keywords

['cat']

In [25]:
from ultralytics import YOLO

# Load a model
model = YOLO('/content/drive/MyDrive/Kursach/yolov8m.pt')  # load an official model


**Пример 1**.(один объект по запросу)
# **Find the cat on this image**



In [77]:
request = "Find the cat on this image"

image = 'Example_1.jpeg'

In [66]:
results = model(image, save = True)  # predict on an image


image 1/1 /content/drive/MyDrive/Kursach/Example_1.jpeg: 448x640 1 cat, 1 dog, 1 bowl, 5 books, 1065.8ms
Speed: 5.2ms preprocess, 1065.8ms inference, 1.4ms postprocess per image at shape (1, 3, 448, 640)
Results saved to runs/detect/predict6


In [80]:

k = 0
for r in results:
  for word in filtered_keywords:
      boxes = r.boxes
      for box in boxes:
          c = box.cls
          if word == model.names[int(c)]:
            print("The object", model.names[int(c)], "detected on", box.xyxy.squeeze().tolist())
            k += 1

if k == 0:
  print("\nObject ", filtered_keywords, " not detected")

The object cat detected on [845.5067138671875, 322.81951904296875, 1622.8408203125, 1067.642578125]


In [ ]:
#Пример для "ручного" построения bbox найденного(искомого) объекта
import cv2

# Загрузка изображения
image = cv2.imread('Example_1.jpeg')

# Координаты прямоугольника (в формате [x_min, y_min, x_max, y_max])
rectangle_coordinates = [845.5067138671875, 322.81951904296875, 1622.8408203125, 1067.642578125]

# Извлечение координат
x_min, y_min, x_max, y_max = rectangle_coordinates

# Нарисовать прямоугольник на изображении
cv2.rectangle(image, (int(x_min), int(y_min)), (int(x_max), int(y_max)), (0, 255, 0), 2)

# Отобразить изображение с прямоугольником
cv2.imwrite('image_with_bbox.jpg', image)



**Результат** для примера 1:

The object cat detected on [845.5067138671875, 322.81951904296875, 1622.8408203125, 1067.642578125]



**Пример 2.**
(несколько объектов по запросу)
# **Find the cat and the dog on this picture**



In [83]:
request = "Find the cat and the dog on this picture"

image = 'Example_1.jpeg'

results = model(image, save = True)  # predict on an image

start_time = time.time()
keywords = kw_model.extract_keywords(request, keyphrase_ngram_range=(1, 1), stop_words = stop_words_list)
end_time = time.time()
# Время в миллисекундах
execution_time = (end_time - start_time) * 1000
# Формирование строки с ключевыми словами, разделенными запятыми

filtered_keywords = [word for word, score in keywords if score > 0.35]

#keyword_string = ','.join([word for word, score in filtered_keywords])




image 1/1 /content/drive/MyDrive/Kursach/Example_1.jpeg: 448x640 1 cat, 1 dog, 1 bowl, 5 books, 1091.4ms
Speed: 4.2ms preprocess, 1091.4ms inference, 1.8ms postprocess per image at shape (1, 3, 448, 640)
Results saved to runs/detect/predict6


In [82]:
filtered_keywords

['cat', 'dog']

In [84]:

k = 0
for r in results:
  for word in filtered_keywords:
      boxes = r.boxes
      for box in boxes:
          c = box.cls
          if word == model.names[int(c)]:
            print("The object", model.names[int(c)], "detected on", box.xyxy.squeeze().tolist())
            k += 1

if k == 0:
  print("\nObject ", filtered_keywords, " not detected")

The object cat detected on [845.5067138671875, 322.81951904296875, 1622.8408203125, 1067.642578125]
The object dog detected on [8.321319580078125, 235.8555908203125, 1088.4532470703125, 1106.7674560546875]


**Результат** для примера 2:

The object cat detected on [845.5067138671875, 322.81951904296875, 1622.8408203125, 1067.642578125]


The object dog detected on [8.321319580078125, 235.8555908203125, 1088.4532470703125, 1106.7674560546875]



**Пример 3.** (объект из запроса не найден на изображении)

# "**Locate the man on this image**"

In [90]:
request = "Locate the man on this image"

image = 'Example_1.jpeg'

results = model(image, save = True)  # predict on an image

start_time = time.time()
keywords = kw_model.extract_keywords(request, keyphrase_ngram_range=(1, 1), stop_words = stop_words_list)
end_time = time.time()
# Время в миллисекундах
execution_time = (end_time - start_time) * 1000
# Формирование строки с ключевыми словами, разделенными запятыми

filtered_keywords = [word for word, score in keywords if score > 0.3]

#keyword_string = ','.join([word for word, score in filtered_keywords])




image 1/1 /content/drive/MyDrive/Kursach/Example_1.jpeg: 448x640 1 cat, 1 dog, 1 bowl, 5 books, 1057.7ms
Speed: 3.4ms preprocess, 1057.7ms inference, 1.3ms postprocess per image at shape (1, 3, 448, 640)
Results saved to runs/detect/predict6


In [91]:
filtered_keywords

['man']

In [92]:

k = 0
for r in results:
  for word in filtered_keywords:
      boxes = r.boxes
      for box in boxes:
          c = box.cls
          if word == model.names[int(c)]:
            print("The object", model.names[int(c)], "detected on", box.xyxy.squeeze().tolist())
            k += 1

if k == 0:
  print("\nObject ", filtered_keywords, " not detected")


Object  ['man']  not detected


Результат для примера 2:

Object  ['man']  not detected